### This notebook produces figures from Ivankovic *et al.*

In [1]:
from inspect import isfunction, ismethod, isgeneratorfunction, isgenerator, isroutine
from inspect import isabstract, isclass, ismodule, istraceback, isframe, iscode, isbuiltin
from inspect import ismethoddescriptor, isdatadescriptor, isgetsetdescriptor, ismemberdescriptor
from inspect import isawaitable, iscoroutinefunction, iscoroutine

from collections.abc import Iterable as iterable

import pickle

def isfx(field): return ismethod(field) or isfunction(field)

class GhostSet:
    """ enhanced interface (ghost) to retrieve class fields """
    def _meta(data): return {k:v for k,v in data.__dict__.items() if not isfx(v)}
    def _at_last(_, sets): pass
    def _set(object, **sets):
        ''' use to fast initialize fields | needed to avoid initialization problems at copy by value '''
        for field in sets: setattr(object, field, sets[field])
        object._at_last(sets)
GSet = GhostSet

def meta(object):
    ''' retrieves clonable object metadata (__dict__) as a copy '''
    if isinstance(object, GSet): return object._meta()
    return {}

class ClonableObjectGhost:
    """ enhanced interface (ghost) for clonable objects """
    def _by_val(_, depth=-1, _layer=0): pass
GCo = ClonableObjectGhost

class ClonableObject(GSet, GCo):
    """ base clonable object """
    def __init__(this, **data): this._set(**data)
    def __call__(_, **options): _._set(**options)
    def _by_val(_, depth=-1, _layer=0):
        copy = type(_)()
        copy._set(**_._meta())
        if depth<0 or depth>_layer:
            for field in copy.__dict__:
                if isinstance(copy.__dict__[field], ClonableObjectGhost):
                    copy.__dict__[field] = copy.__dict__[field]._by_val(depth,_layer+1)
        return copy
COb = ClonableObject

def copy_by_val(object, depth=-1, _layer=0):
    if isinstance(object, GCo): return object._by_val(depth,_layer)
    return object
copy = by_val = vof = copy_by_val

class ComparableGhost:
    """ enhanced interface (ghost) for comparing instances """
    def _compare(a, b):
        if type(a) != type(b): return False
        if a.__dict__ == b.__dict__: return True
        return False
    def __eq__(a, b): return a._compare(b)
GEq = ComparableGhost

class IterableObjectGhost(GSet):
    """ enhanced interface (ghost) for iterables: exposes __dict__,
        therefore Iterable Objects are like lua dictionaries """
    def __contains__(this, key): return key in this.__dict__
    def __iter__(this): return iter(this.__dict__)
    def items(my): return my.__dict__.items()
    def __getitem__(by, field): return by.__dict__[field]
    def __setitem__(by, field, value): by.__dict__[field] = value
    def pop(by, field): return by.__dict__.pop(field)
GIo = IterableObjectGhost

class ReprGhost:
    """ enhanced interface (ghost) for the skeleton method _repr,
        see implementation of Struct for a working example;
        Record __repr__ override uses _lines_ for max lines display """
    _lines_ = 31
    _chars_ = 13
    _msgsz_ = 62
    _ellipsis_ = ' ... '
    def _repr(my, value):
        _type = ''.join(''.join(str(type(value)).split('class ')).split("'"))
        _value = '{}'.format(value)
        if len(_value)>my._chars_:
            show = int(my._chars_/2)
            _value = _value[:show]+my._ellipsis_+_value[-show:]
        return '{} {}'.format(_type, _value)
    def _resize(this, message, at=.7):
        if len(message)>this._msgsz_:
            start = int(at*this._msgsz_)
            end = this._msgsz_-start
            return message[:start]+this._ellipsis_+message[-end:]
        return message
GRe = ReprGhost

def set_repr_to(lines): GRe._lines_ = lines

class Struct(COb, GEq, GIo, GRe):
    """ structured autoprintable object, behaves like a lua dictionary """
    def __repr__(_):
        return '\n'.join(['{}:\t{}'.format(k, _._repr(v)) for k,v in _.items()])
struct = Struct

class RecordableGhost:
    """ enhanced interface (ghost) for type recording,
        see Record for a working example """
    @staticmethod
    def load(filename):
        with open(filename, 'rb') as file: return pickle.load(file)
    def save(data, filename):
        with open(filename, 'wb') as file: pickle.dump(data, file)
        
GRec = RecordableGhost

class Record(GSet, GCo, GRec, GEq, GRe):
    """ wrapper for any object or value, auto-inspects and provides load/save type structure """
    data = None
    _check = dict(
            isfunction=isfunction, ismethod=ismethod, isgeneratorfunction=isgeneratorfunction, isgenerator=isgenerator, isroutine=isroutine,
            isabstract=isabstract, isclass=isclass, ismodule=ismodule, istraceback=istraceback, isframe=isframe, iscode=iscode, isbuiltin=isbuiltin,
            ismethoddescriptor=ismethoddescriptor, isdatadescriptor=isdatadescriptor, isgetsetdescriptor=isgetsetdescriptor, ismemberdescriptor=ismemberdescriptor,
            isawaitable=isawaitable, iscoroutinefunction=iscoroutinefunction, iscoroutine=iscoroutine
                   )
    def __init__(this, token, **meta):
        this.data = token
        this.__dict__.update({k:v(token) for k,v in this._check.items()})
        super()._set(**meta)
    @property
    def type(_): return type(_.data)
    def inherits(_, *types): return issubclass(_.type, types)
    @property
    def isbaseiterable(_): return _.inherits(tuple, list, dict, set) or _.isgenerator or _.isgeneratorfunction
    @property
    def isiterable(_): return isinstance(_.data, iterable) and _.type is not str
    def _clone_iterable(_):
        if _.inherits(dict): return _.data.copy()
        elif _.isgenerator or _.isgeneratorfunction: return (i for i in list(_.data))
        else: return type(_.data)(list(_.data)[:])
    def _meta(data): return {k:v for k,v in data.__dict__.items() if k != 'data' and not isfx(v)}
    def _by_val(_, depth=-1, layer=0):
        data = _.data
        if _.isiterable: data = _._clone_iterable()
        elif _.inherits(ClonableObjectGhost): data = by_val(data, depth, layer)
        return type(_)(data, **meta(_))
    def __enter__(self): self._instance = self; return self
    def __exit__(self, type, value, traceback): self._instance = None
    def __repr__(self):
        if not hasattr(self, '_preprint'): return Record(self.data, _preprint='', _lines=Record(Record._lines_)).__repr__()
        if self.isbaseiterable:
            pre, repr = self._preprint, ''
            for n,i in enumerate(self.data):
                if self._lines.data == 0: break
                else: self._lines.data -= 1
                index, item = str(n), i
                if self.inherits(dict): index += ' ({})'.format(str(i)); item = self.data[i]
                repr += pre+'{}: '.format(index)
                next = Record(item, _preprint=pre+'\t', _lines=self._lines)
                if next.isiterable: repr += '\n'
                repr += next.__repr__()
                repr += '\n'
            return repr
        elif self.inherits(GCo): return Record(self.data._meta(), _preprint=self._preprint, _lines=self._lines).__repr__()
        else: return self._repr(self.data)
REc = Record

class Bisect(list, COb):
    """ bisect implementation using clonable objects """
    def __init__(set, *items, key=None, reverse=False):
        if not key: key = lambda  x:x
        super().__init__(sorted(items, reverse=reverse, key=key))
    def _bisect(set, item, key, reverse, bottom, top):
        def _(check):
            if key: return key(check)
            return check
        at = int((top-bottom)/2)+bottom
        if len(set)==0: return (0,-1)
        if item==_(set[at]): return (at,0)
        bigger = item<_(set[at])
        if bigger != reverse:
            if at-bottom>0: return set._bisect(item, key, reverse, bottom, at)
            return (at,-1)
        elif top-at>1: return set._bisect(item, key, reverse, at, top)
        return (at,1)
    def search(_, item, key=None, reverse=False):
        if not key: key = lambda x:x
        return _._bisect(item, key, reverse, 0, len(_))
    def _by_val(_, depth=-1, _layer=0):
        copy = super()._by_val(depth, _layer)
        copy += _[:]
        return copy
BSx = Bisect

In [2]:
from os import listdir
import pandas as pd

In [3]:
main_folder = "/home/kivi/gdrive/epigame-folder/"
path_net = main_folder + "selected_network/"

In [4]:
# resection_str = input("Define resection nodes \n(Instruction: Format node labels and divide by a comma and a space, as **'N1-N2', 'N2-N3'**):")

# resection = list(resection_str.split(", "))

In [5]:
resection = ['A3-A4', 'B2-B3', 'B3-B4', 'B5-B6', 'B6-B7', 'D3-D4', 'D4-D5', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'T10-T11', 'T11-T12']

In [6]:
df_rows = []

for file_net in listdir(path_net):

  subject_id = file_net.split("/")[-1][0:3]
  print("\nSelected network of", file_net)

  # Load attributes from filename "SUB-woi-connectivitymeasure-(fmin,fmax).res"
  woi = file_net.split("-")[1]
  conn_measure = file_net.split("-")[2]

  # If the signal was not filtered in any specific frequency band, mark it as "NA"
  # If the signal was filtered, mark the frequency band minimum and maximum
  f_min = "NA" if file_net.split(".")[0].split("-")[-1] == "w" else file_net.split(".")[0].split("-")[3][1:-1].split(",")[0]
  f_max = "NA" if file_net.split(".")[0].split("-")[-1] == "w" else file_net.split(".")[0].split("-")[3][1:-1].split(",")[1]

  net = REc.load(path_net + file_net).data

  last_tested_n = list(net.test_nets.keys())[-1]
  print(last_tested_n)
  if last_tested_n>2: print(f"\nEvaluation score decreased at {last_tested_n} nodes:", net.test_nets[last_tested_n-1][0][-1], ">=", net.test_nets[last_tested_n][0][-1])

  eval_progression = [net.test_nets[n][0][-1] for n in net.test_nets.keys()]
  print("Evaluation score progression:", eval_progression)
  eval_score = net.test_nets[last_tested_n-1][0][-1]

  selected = list(net.nodes)
  net_size = len(selected)
  best_pair = sorted(set([node for node in net.test_nets[2][0][1].split("<->")]))

  print("\nOriginal resection:", resection)
  print("Selected net:", selected)

  intersection = list(set(selected)&set(resection))
  print("\nNet-Resection intersection:", intersection)
  val_score = len(intersection) / (net_size*len(resection))
  print("Validation score =", val_score)

  df_rows.append([woi, conn_measure, f_min, f_max, resection, best_pair, selected, net_size, intersection, eval_score, val_score])

dataframe = pd.DataFrame(df_rows, columns=["WOI","CM","F_MIN","F_MAX","RESECTION","BEST_PAIR","NET","NET_SIZE","INTERSECTION","EVAL_SCORE","VAL_SCORE"]) 



Selected network of ASJ-preseizure5-CC-(0,4).res
6

Evaluation score decreased at 6 nodes: 0.9621380846325167 >= 0.9525909592061742
Evaluation score progression: [0.9250535331905779, 0.9340540540540541, 0.9432314410480349, 0.9621380846325167, 0.9525909592061742]

Original resection: ['A3-A4', 'B2-B3', 'B3-B4', 'B5-B6', 'B6-B7', 'D3-D4', 'D4-D5', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'T10-T11', 'T11-T12']
Selected net: ['A10-A11', 'B1-B2', 'D3-D4', 'F9-F10', 'J1-J2']

Net-Resection intersection: ['D3-D4']
Validation score = 0.011111111111111112

Selected network of ASJ-preseizure5-CC-(4,8).res
3

Evaluation score decreased at 3 nodes: 0.9330453563714902 >= 0.9162248144220574
Evaluation score progression: [0.9330453563714902, 0.9162248144220574]

Original resection: ['A3-A4', 'B2-B3', 'B3-B4', 'B5-B6', 'B6-B7', 'D3-D4', 'D4-D5', 'T1-T2', 'T2-T3', 'T3-T4', 'T4-T5', 'T5-T6', 'T6-T7', 'T7-T8', 'T8-T9', 'T9-T10', 'T10-T11', 'T11-T12']
Selected net

In [7]:
dataframe

,WOI,CM,F_MIN,F_MAX,RESECTION,BEST_PAIR,NET,NET_SIZE,INTERSECTION,EVAL_SCORE,VAL_SCORE
0,preseizure5,CC,0,4,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[D3-D4, J1-J2]","[A10-A11, B1-B2, D3-D4, F9-F10, J1-J2]",5,[D3-D4],0.962138,0.011111
1,preseizure5,CC,4,8,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[A9-A10, D4-D5]","[A9-A10, D4-D5]",2,[D4-D5],0.933045,0.027778
2,preseizure5,CC,8,12,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[L5-L6, T8-T9]","[L5-L6, T8-T9]",2,[T8-T9],0.962889,0.027778
3,preseizure5,CC,12,30,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[E7-E8, Q2-Q3]","[B2-B3, E7-E8, Q2-Q3]",3,[B2-B3],0.981818,0.018519
4,preseizure5,CC,30,70,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[D2-D3, E10-E11]","[D2-D3, E10-E11]",2,[],0.962889,0.000000
5,preseizure5,CC,70,150,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[L6-L7, M1-M2]","[L6-L7, M1-M2, P9-P10, T1-T2]",4,[T1-T2],0.972973,0.013889
6,preseizure4,CC,0,4,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[J13-J14, U6-U7]","[J13-J14, P8-P9, Q3-Q4, U6-U7]",4,[],0.925054,0.000000
7,preseizure4,CC,4,8,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[B2-B3, D6-D7]","[B2-B3, D6-D7]",2,[B2-B3],0.919037,0.027778
8,preseizure4,CC,8,12,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[G11-G12, L1-L2]","[G11-G12, L1-L2]",2,[],0.934054,0.000000
9,preseizure4,CC,12,30,"[A3-A4, B2-B3, B3-B4, B5-B6, B6-B7, D3-D4, D4-...","[D1-D2, D4-D5]","[D1-D2, D4-D5]",2,[D4-D5],0.918033,0.027778
